<a href="https://colab.research.google.com/github/as3091/IITJ/blob/NER/ML/Assign_2/NER/NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Title: Named Entity Recognition

Description:
In this NER-focused project, you will design and develop a custom Named Entity Recognition (NER) system for text analysis. Named Entity Recognition involves identifying and classifying specific entities, such as names, dates, locations, and more, within unstructured text data. Your project will offer a versatile NER solution that will work well on the provided dataset.

Dataset: Named Entity Recognition (NER) Corpus (kaggle.com)

https://www.kaggle.com/datasets/naseralqaydeh/named-entity-recognition-ner-corpus

In [1]:
# !pip install tensorflow --quiet
# !pip install keras --quiet

In [2]:
import warnings,sys, ast, pickle
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import tensorflow as tf
from IPython.display import display, HTML
# import matplotlib.pyplot as plt
from pprint import pprint

from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from keras.layers import Dense, Input, Bidirectional, LSTM, Embedding, Dropout
from keras.models import Model
from keras.losses import SparseCategoricalCrossentropy
from keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
# from keras.random import SeedGenerator

# seed_gen = SeedGenerator(seed=42)
tf.random.set_seed(42)
np.random.seed(42)

import datetime as dt

2025-04-13 12:52:30.792704: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-13 12:52:30.812073: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-13 12:52:30.960820: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-13 12:52:31.106999: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744548751.294798   11599 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744548751.33

In [3]:
# !pip install kagglehub --quiet

In [4]:
# while True:
try:
    NER_df = pd.read_csv("ner.csv")
except FileNotFoundError:
    import kagglehub
    from kagglehub import KaggleDatasetAdapter

    # Set the path to the file you'd like to load
    file_path = "ner.csv"

    # Load the latest version
    NER_df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "naseralqaydeh/named-entity-recognition-ner-corpus",
    file_path,

    )

In [5]:
display(NER_df.head(2).T)

,0,1
Sentence #,Sentence: 1,Sentence: 2
Sentence,Thousands of demonstrators have marched throug...,Families of soldiers killed in the conflict jo...
POS,"['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'...","['NNS', 'IN', 'NNS', 'VBN', 'IN', 'DT', 'NN', ..."
Tag,"['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', '...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."


Essential info about entities:

- geo = Geographical Entity
- org = Organization
- per = Person
- gpe = Geopolitical Entity
- tim = Time indicator
- art = Artifact
- eve = Event
- nat = Natural Phenomenon

### Class it

In [6]:
class The_Neural_Net:
    def __init__(self):
        self.max_len = 0

1. Read data

In [7]:
def read_data(self):
    try:
        NER_df = pd.read_csv("ner.csv")
    except FileNotFoundError:
        import kagglehub
        from kagglehub import KaggleDatasetAdapter

        file_path = "ner.csv"

        NER_df = kagglehub.load_dataset(
        KaggleDatasetAdapter.PANDAS,
        "naseralqaydeh/named-entity-recognition-ner-corpus",
        file_path,

        )
    print(NER_df.shape,"\n")
    print("\n",NER_df.info())
    return NER_df
The_Neural_Net.read_data = read_data

2. Preprocess

In [8]:
def PreProcess(self):
    NER_df = self.read_data()
    NER_df.dropna(inplace=True)
    NER_df.drop(columns=["Sentence #","POS"],inplace=True)
    NER_df["Tag"] = NER_df["Tag"].apply(lambda x: ast.literal_eval(x))

    self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(NER_df["Sentence"], NER_df["Tag"], shuffle=True,test_size=0.20, random_state=42)
    del NER_df

    self.X_tokenizer = Tokenizer(lower=False,oov_token="UNK")
    self.X_tokenizer.fit_on_texts(self.X_train)

    self.X_train = self.X_tokenizer.texts_to_sequences(self.X_train)
    self.X_test = self.X_tokenizer.texts_to_sequences(self.X_test)

    self.vocab_len = len(self.X_tokenizer.word_index)
    print(f"Number of unique tokens:\t{self.vocab_len}")

    self.y_tokenizer = Tokenizer(lower=False,oov_token="UNK")
    self.y_tokenizer.fit_on_texts(self.y_train)

    self.y_train = self.y_tokenizer.texts_to_sequences(self.y_train)
    self.y_test = self.y_tokenizer.texts_to_sequences(self.y_test)

    for dataset in [self.X_train,self.X_test]:
        for i in range(len(dataset)):
            self.max_len = max(self.max_len,len(dataset[i]))

    self.X_train = pad_sequences(self.X_train, maxlen=self.max_len, padding="post", value=0)
    self.X_test = pad_sequences(self.X_test, maxlen=self.max_len, padding="post", value=0)

    self.y_train = pad_sequences(self.y_train, maxlen=self.max_len, padding="post", value=0)
    self.y_test = pad_sequences(self.y_test, maxlen=self.max_len, padding="post", value=0)

    self.Number_of_classes_K = len(self.y_tokenizer.word_index) + 1

The_Neural_Net.PreProcess = PreProcess

In [28]:
def model_arch(self):
    vector_size = 128

    input_layer = Input(shape=(self.max_len,))
    embedding_layer = Embedding(input_dim=self.vocab_len + 1, output_dim=vector_size, mask_zero=True, trainable=True)(input_layer)
    dropout_layer_1 = Dropout(0.075)(embedding_layer)
    bidirectional_LSTM_Layer = Bidirectional(LSTM(vector_size * 2, return_sequences=True))(dropout_layer_1)
    output_layer = Dense(self.Number_of_classes_K)(bidirectional_LSTM_Layer)

    self.model = Model(input_layer, output_layer)
    print(self.model.summary())

    self.model.compile(optimizer="adam",loss=SparseCategoricalCrossentropy(from_logits=True),metrics=["accuracy"])
The_Neural_Net.model_arch = model_arch

In [29]:
def model_fit(self,num_of_epochs):
    early_stopping = EarlyStopping(
    monitor='val_loss',  # Metric to monitor (e.g., validation loss)
    patience=3,          # Number of epochs with no improvement after which training will stop
    restore_best_weights=True  # Restore the weights of the best epoch
    )
    self.num_of_epochs = num_of_epochs
    self.model.fit(
            self.X_train,
            self.y_train,
            epochs=num_of_epochs,
            validation_data=(self.X_test, self.y_test),
           # callbacks=[early_stopping]  # Include EarlyStopping in callbacks
        )
The_Neural_Net.model_fit = model_fit


In [30]:
def save_to_file(self):
    with open('X_tokenizer.pkl', 'wb') as file:
        pickle.dump(self.X_tokenizer, file)
    print("X_tokenizer saved to X_tokenizer.pkl")

    with open('y_tokenizer.pkl', 'wb') as file:
        pickle.dump(self.y_tokenizer, file)
    print("y_tokenizer saved to y_tokenizer.pkl")

    model_save_path = f"ner_model_{self.num_of_epochs}.keras"
    # print(model_save_path)
    self.model.save(model_save_path)
    print(f"Model saved to {model_save_path}")
The_Neural_Net.save_to_file = save_to_file

In [31]:
def load_from_file(self,num_of_epochs):
    # self.num_of_epochs = num_of_epochs
    with open('X_tokenizer.pkl', 'rb') as file:
        self.X_tokenizer = pickle.load(file)
        print("Tokenizer loaded from X_tokenizer.pkl")

    with open('y_tokenizer.pkl', 'rb') as file:
        self.y_tokenizer = pickle.load(file)
        print("Tokenizer loaded from y_tokenizer.pkl")

    model_save_path = f"ner_model_{num_of_epochs}.keras"
    self.model = load_model(model_save_path)
    print(f"Model loaded from {model_save_path}")


The_Neural_Net.load_from_file = load_from_file

In [32]:
def predict(self,model,sentence):
    sentence_tokens = self.X_tokenizer.texts_to_sequences([sentence])
    # print(len(sentence.split()))
    # print(len(sentence_tokens[0]))
    # tokens_to_words = [word for word, index in self.X_tokenizer.word_index.items() if index in sentence_tokens[0]]

    predictions = model.predict(pad_sequences(sentence_tokens,
                                            maxlen=self.max_len,
                                            padding="post"))
    # print(predictions)
    prediction_ner = np.argmax(predictions,axis=-1)
    # print(prediction_ner)

    NER_tags = [self.y_tokenizer.index_word[num] for num in list(prediction_ner.flatten())]
    final_pred = {"Word":[],"Tag":[]}
    sentence_split = sentence.split()
    for Word,Tag in zip(sentence_split,NER_tags):
        # final_pred[tokens_to_words[i]] = NER_tags[i]
        final_pred["Word"].append(Word)
        final_pred["Tag"].append(Tag)
    return pd.DataFrame(final_pred)
The_Neural_Net.predict = predict

In [33]:
NN_obj = The_Neural_Net()
NN_obj.PreProcess()

(47959, 4) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47959 entries, 0 to 47958
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Sentence #  47959 non-null  object
 1   Sentence    47959 non-null  object
 2   POS         47959 non-null  object
 3   Tag         47959 non-null  object
dtypes: object(4)
memory usage: 1.5+ MB

 None
Number of unique tokens:	28761


In [ ]:
num_of_epochs = 50
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='local') # Detect TPU
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    tpu_strategy = tf.distribute.TPUStrategy(tpu)

    num_of_epochs = 100
    with tpu_strategy.scope():
        NN_obj.model_arch()
        NN_obj.model_fit(num_of_epochs)

except Exception as e:
    NN_obj.model_arch()
    NN_obj.model_fit(num_of_epochs)


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 89)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 89, 128)   │  3,681,536 │ input_layer_2[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 89, 128)   │          0 │ embedding_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_3         │ (None, 89)        │          0 │ input_layer_2[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_2     │ (None, 89, 512)   │    788,480 │ dropout_2[0][0],  │
│ (Bidirectional)     │                   │            │ not_equal_3[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 89, 19)    │      9,747 │ bidirectional_2[… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,479,763 (17.09 MB)

 Trainable params: 4,479,763 (17.09 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/50
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 307s 254ms/step - accuracy: 0.2160 - loss: 0.6814 - val_accuracy: 0.2206 - val_loss: 0.3185
Epoch 2/50
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 284s 237ms/step - accuracy: 0.2232 - loss: 0.2918 - val_accuracy: 0.2234 - val_loss: 0.2809
Epoch 3/50
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 288s 240ms/step - accuracy: 0.2274 - loss: 0.2247 - val_accuracy: 0.2244 - val_loss: 0.2773
Epoch 4/50
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 278s 232ms/step - accuracy: 0.2305 - loss: 0.1859 - val_accuracy: 0.2247 - val_loss: 0.2909
Epoch 5/50
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 284s 237ms/step - accuracy: 0.2329 - loss: 0.1556 - val_accuracy: 0.2246 - val_loss: 0.3029
Epoch 6/50
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 276s 230ms/step - accuracy: 0.2347 - loss: 0.1339 - val_accuracy: 0.2243 - val_loss: 0.3188
Epoch 7/50
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 298s 249ms/step - accuracy: 0.2361 - loss: 0.1150 - val_accuracy: 0.2244 - val_loss: 0.3427
Epoch 8/50
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 303s 252ms/step

In [ ]:
# num_of_epochs = 5
# if tf.test.is_gpu_available():
#     num_of_epochs = 100
#     with tf.device('/device:GPU:0'):
#         NN_obj.model_fit(num_of_epochs)
# else:
#     NN_obj.model_fit(num_of_epochs)


In [23]:
NN_obj.save_to_file()

X_tokenizer saved to X_tokenizer.pkl
y_tokenizer saved to y_tokenizer.pkl
Model saved to ner_model_50.keras


In [24]:
NN_obj.load_from_file(num_of_epochs=num_of_epochs)

Tokenizer loaded from X_tokenizer.pkl
Tokenizer loaded from y_tokenizer.pkl
Model loaded from ner_model_50.keras


In [25]:
sentence = """Is this the real life? Is this just fantasy? Caught in a landslide, no escape from reality"""
model = NN_obj.model
prediction_df = NN_obj.predict(model=NN_obj.model,sentence=sentence)
display(prediction_df)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 320ms/step


,Word,Tag
0,Is,O
1,this,O
2,the,O
3,real,O
4,life?,O
5,Is,O
6,this,O
7,just,O
8,fantasy?,O
9,Caught,O


In [26]:
sentence = """Apoorv Code, Ankur question one, Alok, zoom meeting"""
model = NN_obj.model
prediction_df = NN_obj.predict(model=NN_obj.model,sentence=sentence)
display(prediction_df)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step


,Word,Tag
0,Apoorv,O
1,"Code,",O
2,Ankur,O
3,question,O
4,"one,",O
5,"Alok,",O
6,zoom,I-per
7,meeting,O


# Rough work below this

In [ ]:
# Save the trained model to a file
# dt_now = dt.datetime.now().strftime("%y%m%d_%H%M")
model_save_path = f"ner_model_{num_of_epochs}.keras"  # You can specify any file path
# print(model_save_path)
NN_obj.model.save(model_save_path)
print(f"Model saved to {model_save_path}")

In [ ]:
loaded_model = load_model(model_save_path)

sentence = """Is this the real life? Is this just fantasy? Caught in a landslide, no escape from reality"""

prediction_df = NN_obj.predict(model=loaded_model,sentence=sentence)
display(prediction_df)

In [ ]:
sys.exit()

# Rough work below this

In [ ]:
from tensorflow.keras.models import load_model

loaded_model = load_model(model_save_path)

In [ ]:
sys.exit()

In [ ]:
from keras.layers import Dense, Dropout, Input
from keras.models import Model

input_layer = Input(shape=(10,))
dense_layer = Dense(32, activation='relu')(input_layer)
dropout_layer = Dropout(0.5)(dense_layer)
output_layer = Dense(1, activation='sigmoid')(dropout_layer)

model = Model(inputs=input_layer, outputs=output_layer)
model.summary()

3. Tokenize

In [ ]:
def Tokenize(self):



The_Neural_Net.PreProcess = PreProcess

In [ ]:
print(NER_df.shape)

In [ ]:
print(NER_df.info())

In [ ]:
NER_df.dropna(inplace=True)
NER_df.drop(columns=["Sentence #","POS"],inplace=True)
NER_df["Tag"] = NER_df["Tag"].apply(lambda x: ast.literal_eval(x))

In [ ]:
print(NER_df.info())

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(NER_df["Sentence"], NER_df["Tag"], shuffle=True,test_size=0.20, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, shuffle=True,test_size=0.50, random_state=42)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
X_tokenizer = Tokenizer(lower=False,oov_token="UNK")
X_tokenizer.fit_on_texts(X_train)

In [ ]:
X_train.reset_index(drop=True)

In [ ]:
X_train = X_tokenizer.texts_to_sequences(X_train)
X_test = X_tokenizer.texts_to_sequences(X_test)
X_val = X_tokenizer.texts_to_sequences(X_val)

In [ ]:
vocab_len = len(X_tokenizer.word_index)
print(f"Number of unique tokens:\t{vocab_len}")

In [ ]:
train_tags = set([val for sublist in y_train for val in sublist])
test_tags = set([val for sublist in y_test for val in sublist])
val_tags = set([val for sublist in y_val for val in sublist])

print("Unique NER tags in train set: ",train_tags)
print("Unique NER tags in test set: ",test_tags)
print("Unique NER tags in test set: ",val_tags)

In [ ]:
y_tokenizer = Tokenizer(lower=False,oov_token="UNK")
y_tokenizer.fit_on_texts(y_train)

In [ ]:
y_train = y_tokenizer.texts_to_sequences(y_train)
y_test = y_tokenizer.texts_to_sequences(y_test)
y_val = y_tokenizer.texts_to_sequences(y_val)

In [ ]:
max_len = 0
for dataset in [X_train,X_test,X_val]:
    for i in range(len(dataset)):
        max_len = max(max_len,len(dataset[i]))

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train = pad_sequences(X_train, maxlen=max_len, padding="post", value=0)
X_test = pad_sequences(X_test, maxlen=max_len, padding="post", value=0)
X_val = pad_sequences(X_val, maxlen=max_len, padding="post", value=0)

y_train = pad_sequences(y_train, maxlen=max_len, padding="post", value=0)
y_test = pad_sequences(y_test, maxlen=max_len, padding="post", value=0)
y_val = pad_sequences(y_val, maxlen=max_len, padding="post", value=0)

In [ ]:
for dataset in [X_train,X_test,X_val,y_train,y_test,y_val]:
    print(dataset.shape)

In [ ]:
Number_of_classes_K = len(y_tokenizer.word_index) + 1

In [ ]:
from keras.layers import Dense, Dropout, Input
from keras.models import Model

input_layer = Input(shape=(10,))
dense_layer = Dense(32, activation='relu')(input_layer)
dropout_layer = Dropout(0.5)(dense_layer)
output_layer = Dense(1, activation='sigmoid')(dropout_layer)

model = Model(inputs=input_layer, outputs=output_layer)
model.summary()

In [ ]:
from keras.layers import Dense, Input, Bidirectional, LSTM, Embedding, Dropout
from keras.models import Model
from keras.losses import SparseCategoricalCrossentropy
from keras.callbacks import EarlyStopping

In [ ]:
vector_size = 64

input_layer = Input(shape=(max_len,))
embedding_layer = Embedding(input_dim=vocab_len + 1, output_dim=vector_size, mask_zero=True, trainable=True)(input_layer)
dropout_layer_1 = Dropout(0.2)(embedding_layer)
bidirectional_LSTM_Layer = Bidirectional(LSTM(vector_size * 2, return_sequences=True))(dropout_layer_1)
output_layer = Dense(Number_of_classes_K)(bidirectional_LSTM_Layer)

model = Model(input_layer, output_layer)
print(model.summary())

In [ ]:
pprint(model.get_config())

In [ ]:
early_stopping = EarlyStopping(
    monitor='val_loss',  # Metric to monitor (e.g., validation loss)
    patience=3,          # Number of epochs with no improvement after which training will stop
    restore_best_weights=True  # Restore the weights of the best epoch
)
model.compile(optimizer="adam",loss=SparseCategoricalCrossentropy(from_logits=True),metrics=["accuracy"])
model.fit(
    X_train,
    y_train,
    epochs=6,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping]  # Include EarlyStopping in callbacks
)

In [ ]:
sentence = """Is this the real life? Is this just fantasy? Caught in a landslide, no escape from reality"""
unpadded_len = len(sentence.split(" "))
predictions = model.predict(pad_sequences(X_tokenizer.texts_to_sequences([sentence]),
                                          maxlen=max_len,
                                         padding="post"))
print(predictions)

In [ ]:
prediction_ner = np.argmax(predictions,axis=-1)
print(prediction_ner)

In [ ]:
NER_tags = [y_tokenizer.index_word[num] for num in list(prediction_ner.flatten())][:unpadded_len]
print(NER_tags)

In [ ]:
sys.exit()

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

In [ ]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

In [ ]:
tf.test.is_gpu_available()

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

In [ ]:
predictions = model(x_train[:1]).numpy()
predictions

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [ ]:
tf.device('/device:GPU:0')

In [ ]:
with tf.device('/device:GPU:0'):
    model.fit(x_train, y_train, epochs=500, batch_size=32)

In [ ]:
model.evaluate(x_test,  y_test, verbose=2)

In [ ]:
predictions = model(x_test[:1]).numpy()

In [ ]:
predictions